In [48]:
#Used in Tensorflow Model
import numpy as np
import tensorflow as tf
import tflearn
import random

#Usde to for Contextualisation and Other NLP Tasks.
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

#Other
import json
import pickle
import warnings

In [49]:
with open('intentss1.json') as json_data:
    intents = json.load(json_data)

In [50]:
words = []
classes = []
documents = []
ignore_words = ['?']
print("Looping through the Intents to Convert them to words, classes, documents and ignore_words.......")
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

Looping through the Intents to Convert them to words, classes, documents and ignore_words.......


In [51]:
print("Stemming, Lowering and Removing Duplicates.......")
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

Stemming, Lowering and Removing Duplicates.......
38 documents
8 classes ['благодарность', 'жалоба', 'имя', 'о себе', 'помощь', 'приветствие', 'проблема', 'прощание']
83 unique stemmed words ['!', ',', '.', 'аниса', 'будьте', 'вам', 'вас', 'встречи', 'вы', 'день', 'для', 'до', 'добры', 'добрый', 'есть', 'жалоба', 'жалобу', 'за', 'забыл', 'заканчивается', 'здравствуйте', 'здрасьте', 'зовут', 'зовуть', 'и', 'извини', 'имя', 'интернет', 'интернеть', 'к', 'как', 'кто', 'меня', 'мне', 'можете', 'можешь', 'на', 'не', 'нибуд', 'нужна', 'обращаться', 'обслуживания', 'общение', 'огромное', 'ой', 'очень', 'план', 'плохая', 'плохо', 'подать', 'поддержка', 'пожалуйста', 'пока', 'поменять', 'помогите', 'помоч', 'помочь', 'помощь', 'почему', 'привет', 'приятное', 'проблема', 'работает', 'работаеть', 'работе', 'свой', 'связю', 'сделать', 'сменить', 'со', 'создал', 'спасибо', 'тарифный', 'твое', 'тебя', 'трафик', 'ты', 'у', 'удачи', 'хей', 'хочу', 'что', 'я']


In [52]:
print("Creating the Data for our Model.....")
training = []
output = []
print("Creating an List (Empty) for Output.....")
output_empty = [0] * len(classes)

print("Creating Traning Set, Bag of Words for our Model....")
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

Creating the Data for our Model.....
Creating an List (Empty) for Output.....
Creating Traning Set, Bag of Words for our Model....


In [59]:
print(output_row)

[0, 1, 0, 0, 0, 0, 0, 0]


In [53]:
print("Shuffling Randomly and Converting into Numpy Array for Faster Processing......")
random.shuffle(training)
training = np.array(training)

print("Creating Train and Test Lists.....")
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Building Neural Network for Out Chatbot to be Contextual....")
print("Resetting graph data....")

Shuffling Randomly and Converting into Numpy Array for Faster Processing......
Creating Train and Test Lists.....
Building Neural Network for Out Chatbot to be Contextual....
Resetting graph data....


In [54]:
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)
print("Training....")

Training....


In [55]:
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')


In [56]:
print("Training the Model.......")
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
print("Saving the Model.......")
model.save('model.tflearn')


Training the Model.......


IndexError: list index out of range

In [19]:
print("Pickle is also Saved..........")
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )

Pickle is also Saved..........


In [20]:
print("Loading Pickle.....")
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']


with open('intentss.json') as json_data:
    intents = json.load(json_data)
    
print("Loading the Model......")
# load our saved model
model.load('./model.tflearn')

Loading Pickle.....
Loading the Model......
INFO:tensorflow:Restoring parameters from /Users/mac/Desktop/Working Place/Megafon AI chat bot/untitled folder/AI_ChatBot_Python-master/model.tflearn


In [ ]:
def clean_up_sentence(sentence):
    # It Tokenize or Break it into the constituents parts of Sentense.
    sentence_words = nltk.word_tokenize(sentence)
    # Stemming means to find the root of the word.
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# Return the Array of Bag of Words: True or False and 0 or 1 for each word of bag that exists in the Sentence
def bow(sentence, words, show_details=False):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

ERROR_THRESHOLD = 0.25
# print("ERROR_THRESHOLD = 0.25")

def classify(sentence):
    # Prediction or To Get the Posibility or Probability from the Model
    results = model.predict([bow(sentence, words)])[0]
    # Exclude those results which are Below Threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # Sorting is Done because heigher Confidence Answer comes first.
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1])) #Tuppl -> Intent and Probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # That Means if Classification is Done then Find the Matching Tag.
    if results:
        # Long Loop to get the Result.
        while results:
            for i in intents['intents']:
                # Tag Finding
                if i['tag'] == results[0][0]:
                    # Random Response from High Order Probabilities
                    return print(random.choice(i['responses']))

            results.pop(0)
        



In [22]:
while True:
    input_data = input("Вы - ")
    answer = response(input_data)

Вы - Привет
Радa вас видеть!
Вы - Как тебя зовут?
Просто Аниса
Вы - Привет
Привет
Вы - Что ты?
Меня зовут Аниса, я бот созданный на основе Искусственного Интелекта
Вы - Кто тебя создал?
Меня зовут Аниса, я бот созданный на основе Искусственного Интелекта
Вы - Можешь помочь мне?
Да, конечно, Как я могу вас помочь
Вы - У меня интернет плохо работаеть
Проблема наблюдается в базовой станции
Вы - Кто тебя создал?
Меня зовут Аниса, я бот созданный на основе Искусственного Интелекта
Вы - что ты?
я Аниса
Вы - откуда ты?
Я Аниса, меня создали ребята из Академии ИИ zyplai
Вы - знаешь про Таджикистан?
Радa вас видеть!
Вы - Что знашь про Мегафон
Был рад поболтать с вами
Вы - Как ты?
Просто Аниса
Вы - Сколько тебе лет?
Привет
Вы - хей
Слущаю
Вы - пока
Спасибо за обрашение! Приходите еще раз буду рада помочь вам
Вы - Аниса
Пожалуйста, предоставьте нам вашу жалобу, чтобы помочь вам
Вы - Аниса
Пожалуйста, предоставьте нам вашу жалобу, чтобы помочь вам
Вы - Что ты?
я Аниса
Вы - и
 Как я могу вам помочь

KeyboardInterrupt: Interrupted by user